##This is a demo of tha analysis that can be done with the behavioral_syntax Python 3 library which is based upon the ideas paper 'Changes in postural syntax characterize sensory modulation and natural variation of C. Elegans locomotion' co-authored by RF Shwarz, Andre Brown, William Schafer and many others. Here are the main ideas:
1. animal tracking can increasingly be performed automatically but an outstanding challenge is finding ways to represent these behavioral data
2. the following paper considers the idea that locomotion is driven by shape changes coordinated by the nervous system through time
3. At any given moment the current worm posture can be approximated by the closest matching template from a pre-defined set of postures(i.e. kNN was used) obtained using K-means++ clustering. This is analogous to constructing a histogram where discrete bins are used to represent data drawn from a continuous distribution.
4. With 90 template postures it's possible to capture around 83% of the postural variance and this opens up the possibility that NLP and bioinformatic methods can then be used to analyze worm locomotion. 

-some of the above points are taken directly from the above paper. 

###Kmeans++ clustering
1. The similarity measure between postures and skeletons that was used was the tangent angle distance measure. 
2. Non-trivial(and positive) gain in R2(variance explained) between number of templates was used to determine when to stop increasing the number of templates. i.e. the elbow method was used. 
3. sklearn was used for Kmeans++ clustering

In [ ]:
#below is the transformation used to get tangent angles from worm skeletons:

import numpy as np

dX = np.diff(x, n=1, axis=0)
dY = np.diff(y, n=1, axis=0)

# calculate tangent angles.  atan2 uses angles from -pi to pi instead...
# of atan which uses the range -pi/2 to pi/2.
angles = np.arctan2(dY, dX)

In [ ]:
#once we've got all the angle arrays together, we can apply the positive R2 gain decision rule.
#mean_r_sq() is a function called by decision_rule() as many times as the R2 gain is greater 
#than a pre-defined delta(greater than zero). 

import numpy as np
from sklearn import manifold, cluster

def mean_r_sq(k):
    kmeans = cluster.KMeans(init='k-means++',n_clusters=k,max_iter=1000)
    kmeans.fit(angles)
    
    labels = kmeans.labels_
    centroids = kmeans.cluster_centers_
    
    #compute average variance explained:
    total_var = []
    for i in range(len(centroids)):
        total_var+= [np.corrcoef(centroids[i],j)[0][1]**2 for j in angle_0[np.where(labels == i)]]
    
    return [np.mean(total_var), centroids, labels]


# prev = minimum cluster size
# inc = incremental increase in cluster size k
# delta = R_squared gain that is considered negligible
def decision_rule(prev,inc,delta):
    
    while mean_r_sq(prev+inc)[0]-mean_r_sq(prev)[0] > delta:
        prev = prev+inc
        output = mean_r_sq(prev)
    return output

###In the paper, this resulted in the following template postures:
<img src="https://raw.githubusercontent.com/AidanRocke/behavioral_syntax/master/ipython_notebooks/90_postures.png">

And we might be interested in looking at the tangent angle distance between these postures. Ideally
they wouldn't be too close together. This is something we can approximately do with multidimensional 
scaling with the code below. It must be noted that in general for more than 4 points having more than
two dimensions you need at least three dimensions for the distances to be preserved. Hence, for a large
number of points MDS doesn't work very well. But, for the above postures the distances between the postures
was actually preserved quite well. 

In [ ]:
import numpy as np

#Below is the code used for MDS. The Bokeh plotting library is still growing
#fast and interactivity is probably its greatest feature. 
from bokeh.plotting import figure, output_file, show
from bokeh.models import Range1d

from scipy import io
import numpy as np
from bokeh.plotting import figure, show, output_file
from sklearn.metrics import euclidean_distances

#this allows us to view the tangent angle distance between postures which 
#helps to develop an intuition of which postures are close and which
#are far apart. 

g = io.loadmat('/Users/cyrilrocke/Documents/c_elegans/data/postures')
postures = g.get('postures')


seed = np.random.RandomState(seed=3)

similarities = euclidean_distances(postures.T)

mds = manifold.MDS(n_components=2, max_iter=5000, eps=1e-9, random_state=seed,
                   dissimilarity="precomputed", n_jobs=1)
                   
pos = mds.fit(similarities).embedding_

def mtext(p, x, y, textstr):
    p.text(x, y, text=[textstr],
         text_color='steelblue', text_align="center", text_font_size="10pt")


output_file("color_scatter.html", title="postures MDS")
TOOLS="resize,crosshair,pan,wheel_zoom,box_zoom,reset,tap,previewsave,box_select,poly_select,lasso_select"

# create a new plot with a range set with a tuple
p = figure(plot_width=400, plot_height=400, x_range=(-8, 10),y_range=(-8,8),tools=TOOLS)


p.scatter(pos[:,0],pos[:,1],radius=0.2,fill_color='#FFDAB9', fill_alpha=0.6)

for i in range(90):
    mtext(p, [pos[:,0][i]], [pos[:,1][i]-0.15], str(i))

show(p)

####Here's a visualization of the distance between the postures using. The numbers are consistent with the numbers
####given to the posture templates above:
<img src="https://raw.githubusercontent.com/AidanRocke/behavioral_syntax/master/ipython_notebooks/bokeh_postures_mds.png">

Now, it must be noted that on average the variance explained by the best posture matches was quite good as shown by the graphs below. Each graph represents the postural variance explained(R2) during a 15 minute video. In red we have the average value for R2 and at the top of the graph you have an explanation of how often R2 is above the red line(i.e. the average value):

<img src="https://raw.githubusercontent.com/AidanRocke/behavioral_syntax/master/ipython_notebooks/explained_variance_visualized.png">

####But, R2 and the tangent angle distance actually compute very different things and as the of R2 vs tangent angle distance shows below...the relationship isn't linear. In fact their outliers are different:
<img src="https://raw.githubusercontent.com/AidanRocke/behavioral_syntax/master/ipython_notebooks/variance_explained_vs_errors.png">

-R2 outliers tend to be straight whereas tangent angle distance outliers tend to be curved. In fact, I think that using the R2 gain in choosing the number of templates isn't sound in this respect. I would stick with looking at the decline in the average tangent angle distance error. 


What's coming up next week:
    1. Can we use a non-uniform time warping method to compress sequences in the manner {2,3,3,3,4,5,90}={2,3,4,5,90} without significant loss of information. And, does this require the assumption that C Elegans has discrete behavioral states?
    2. Given the same environmental stimulus do a large number of worms of the same subspecies must have a common set of trigrams? Thanks to data that Andre Brown just recently sent over I will be able to answer this question. 